<a href="https://www.kaggle.com/code/dalloliogm/blend-of-top-3-public-solutions-multiple-strateg?scriptVersionId=246482642" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Blend of top solutions


This notebook is based on the amazing notebook https://www.kaggle.com/code/nina2025/blend-of-top-3-public-solutions . Thanks for sharing it!

I've added a few additional strategies for combining the submissions.

This notebook based on data submissions:


- #### LB=0.36995 - v1 - [Ex11 : Org+PSS5E6 : Autogluon : 36 models - 2](https://www.kaggle.com/code/gauravduttakiit/ex11-org-pss5e6-autogluon-36-models-2) by [Gaurav Dutta](https://www.kaggle.com/gauravduttakiit)
- #### LB=0.36964 - v3 - [Optimal Fertilizers | NN+XGB+LGBM+CAT+HGB+YDF](https://www.kaggle.com/code/mikhailnaumov/optimal-fertilizers-nn-xgb-lgbm-cat-hgb-ydf) by [Mikhail Naumov](https://www.kaggle.com/mikhailnaumov)
- #### LB=0.37228 - v1 - [PG S5E6 - XGBx3 - [CV 0.37049 LB 0.37228]](https://www.kaggle.com/code/mahoganybuttstrings/pg-s5e6-xgbx3-cv-0-37049-lb-0-37228) by [Mahog](https://www.kaggle.com/mahoganybuttstrings)



In [1]:
import pandas as pd
from collections import Counter

# Load predictions
path = '/kaggle/input/16-june-2025-fertilizer-18/submission__LB_'

df1  = pd.read_csv(path + '0_36995__v01__Gaurav_Dutta.csv').rename(columns={'Fertilizer Name': 'Gaurav_Dutta'})
df2  = pd.read_csv(path + '0_36964__v03__M_Naumov.csv')     .rename(columns={'Fertilizer Name': 'M_Naumov'})
df3  = pd.read_csv(path + '0_37228__v01__Mahog.csv')        .rename(columns={'Fertilizer Name': 'Mahog'})
df3g = pd.read_csv(path + '0_37228__v01__Mahog__GEN.csv')   .rename(columns={'Fertilizer Name': 'Mahog_GEN'})

# Merge
dfs = df1[['id', 'Gaurav_Dutta']].merge(
    df2[['id', 'M_Naumov']], on='id').merge(
    df3[['id', 'Mahog']], on='id').merge(
    df3g[['id', 'Mahog_GEN']], on='id')

# Utility function
def get_top3(row, method='vote', weights=None):
    preds = [row['Gaurav_Dutta'], row['M_Naumov'], row['Mahog'], row['Mahog_GEN']]
    if method == 'vote':
        return ' '.join([x for x, _ in Counter(preds).most_common(3)])
    
    elif method == 'weighted':
        score_map = {
            row['Gaurav_Dutta']: weights['Gaurav_Dutta'],
            row['M_Naumov']: weights['M_Naumov'],
            row['Mahog']: weights['Mahog'],
            row['Mahog_GEN']: weights['Mahog_GEN']
        }
        sorted_preds = sorted(score_map.items(), key=lambda x: -x[1])
        seen, result = set(), []
        for label, _ in sorted_preds:
            if label not in seen:
                result.append(label)
                seen.add(label)
            if len(result) == 3:
                break
        return ' '.join(result)
    
    elif method == 'consensus_fill':
        counts = Counter(preds)
        common = [x for x, c in counts.items() if c > 1]
        rest = [x for x in preds if x not in common]
        final = common + [x for x in rest if x not in common]
        return ' '.join(dict.fromkeys(final))[:3]  # Remove duplicates
    
    elif method == 'stacked':
        # take each model's first choice in order of assumed LB rank
        ordered = [row['Gaurav_Dutta'], row['M_Naumov'], row['Mahog'], row['Mahog_GEN']]
        return ' '.join(dict.fromkeys(ordered))[:3]
    
    elif method == 'diverse':
        return ' '.join(dict.fromkeys(preds))[:3]

    else:
        raise ValueError("Unknown method")

# Output different ensemble submissions
weights = {'Gaurav_Dutta': 0.9, 'M_Naumov': 0.8, 'Mahog': 0.75, 'Mahog_GEN': 0.7}
methods = ['vote', 'weighted', 'consensus_fill', 'stacked', 'diverse']

for method in methods:
    dfs['Fertilizer Name'] = dfs.apply(get_top3, axis=1, method=method, weights=weights)
    sub = dfs[['id', 'Fertilizer Name']]
    sub.to_csv(f'ensemble_{method}.csv', index=False)
